In [2]:
# install spacy library
%pip install -U spacy

# Install spacy-transformers
%pip install spacy[transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.5 MB/s eta 0:00:00


In [3]:
# Download the traditional spacy english language large model
%%python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 8.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-04-04 11:46:56.418008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 11:46:58.413969: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-04 11:47:02.581827: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
import pandas as pd
#import praw
#import pprint
import numpy as np
import datetime
import string 

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')

from collections import defaultdict

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Import reddit data 

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1sJcOxw9HGa0Fzl7gR81CZSZMPyGqwZ0wCQ57W910BE8/edit#gid=1148002703')

In [7]:
sheet = wb.worksheet('t')

In [8]:
data = sheet.get_all_values()

In [9]:
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df = df[["dataset","text_clean"]]

In [10]:
df

,dataset,text_clean
1,train,people wait for existing faculty to die to hav...
2,train,people who are being layed off are they softwa...
3,train,personally i switched jobs after years of expe...
4,train,pms dont do performance reviews thats for mana...
5,train,pretty hard more like impossible due to legal ...
...,...,...
20554,test,update just got laid off st year ib analyst ju...
20555,test,offer rescinded at large bank was laid off bef...
20556,test,what roles are the first to get laid off in an...
20557,test,what would you do if you were laid off from a ...


## NER

In [12]:
import spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [13]:
upper = ["Russia","NTUC", "PAP", "Hong Kong","HK", "Ukraine","United States","Europe","United Nations","European Union","China","Syria","India","UAE","Singapore"]
lower = ["russia","ntuc", "pap", "hong kong", "hk", "ukraine","united states","europe","united nations","european union","china","syria","india","uea","singapore"]

In [14]:
##Updating vocabularies
nlp = spacy.load('en_core_web_md')
for x,y in zip(upper,lower):
    x = "u'"+x+"'"
    y = "u'"+y+"'"
    a = nlp.vocab[x]
    b = nlp.vocab[y]
    a.is_lower = b.is_lower
    a.shape = b.shape
    a.shape_ = b.shape_
    a.is_upper =b.is_upper
    a.cluster = b.cluster
    a.is_title = b.is_title

In [16]:
df3 = pd.DataFrame(
    {'text': [],
     'tags': []
    }
)

In [18]:
for i in df['text_clean']:
    count = 0 
    text = i
    a = []
    i = nlp(i)
    for word in i.ents:
        x = word.text+'-'+word.label_
        a.append(x)
    df3.loc[len(df3)]=[text,a]

KeyboardInterrupt: ignored

In [ ]:
df3.index = np.arange(1, len(df) + 1)
df3

In [ ]:
df["tags"] = df3["tags"]
df

In [ ]:
result = [ner(text) for text in df['text_clean']]
result

In [ ]:
df.iloc[0]

In [ ]:
count = 0 
for dic in result: 
  if len(dic) > 0:
    for key in dic: 
      df[key].iloc[count] = dic[key]
    count += 1
  else:
    count += 1

df

In [ ]:
df.to_csv("ner2.csv", index=False)